# Python HANA ML APL

<div class="alert alert-block alert-info">
<b>Get JSON code of predictive model for standalone apply.</b> <br>
</div>

## Learn from historical data

### Create an HANA Dataframe for the training data

In [1]:
from hana_ml import dataframe as hd
conn = hd.ConnectionContext(userkey='MLMDA_KEY')
sql_cmd = 'SELECT * FROM "APL_SAMPLES"."AUTO_CLAIMS_FRAUD" ORDER BY CLAIM_ID'
hdf_train = hd.DataFrame(conn, sql_cmd)

In [2]:
hdf_train.head(6).collect()

,CLAIM_ID,DAYS_TO_REPORT,BODILY_INJURY_AMOUNT,PROPERTY_DAMAGE,PREVIOUS_CLAIMS,PAYMENT_METHOD,IS_REAR_END_COLLISION,PREM_AMOUNT,AGE,GENDER,MARITAL_STATUS,INCOME_ESTIMATE,INCOME_CATEGORY,POLICY_HOLDER,IS_FRAUD
0,CL_0000765,8,0,1957,0,CC,No,Safedriving_discount,52,Male,Single,15906.0,15,Y,No
1,CL_0000832,30,2541,3843,0,CC,No,Safedriving_discount,85,Female,Single,91100.6,75,N,No
2,CL_0002015,4,0,25719,0,CC,No,Standard,45,Male,Married,67085.4,50,N,No
3,CL_0002854,0,0,83,1,Auto,No,Standard,75,Male,Married,49561.0,35,N,No
4,CL_0002869,22,0,1264,0,CC,Yes,Standard,48,Female,Married,39598.7,35,N,No
5,CL_0003400,3,9903,7333,0,Auto,No,Safedriving_discount,41,Male,Single,48271.8,35,N,Yes


### Fit with APL Gradient Boosting

In [3]:
from hana_ml.algorithms.apl.gradient_boosting_classification import GradientBoostingBinaryClassifier
apl_model = GradientBoostingBinaryClassifier()
apl_model.set_params(other_train_apl_aliases={'APL/VariableAutoSelection':'true'})
apl_model.fit(hdf_train, label='IS_FRAUD', key='CLAIM_ID')

In [4]:
my_filter = "\"Partition\" = 'Validation' and \"Indicator\" in ('AUC','F1 Score','Cohen''s kappa')"
df = apl_model.get_debrief_report('ClassificationRegression_Performance').filter(my_filter).collect()
df.drop('Oid', axis=1, inplace=True)
format_dict = {'Value':'{:,.3f}'}
df.style.format(format_dict).hide_index()

Target,Partition,Indicator,Value
IS_FRAUD,Validation,F1 Score,0.595
IS_FRAUD,Validation,AUC,0.923
IS_FRAUD,Validation,Cohen's kappa,0.543


### Export Apply Code

In [5]:
apl_scoring_equation = apl_model.export_apply_code(code_type='JSON')

In [6]:
text_file = open("apl_model.json", "w")
text_file.write(apl_scoring_equation)
text_file.close()